In [1]:
import pymysql
import json
import matplotlib.pyplot as plt
from functools import partial
import csv
import networkx as nx
import freeman as fm
from random import random, choices
def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [2]:
def add_node(g, n):
    try:
        g.add_node(n)
        g.nodes[n]['pos'] = (random(), random())
        g.nodes[n]['label'] = n
        g.nodes[n]['color'] = (255, 0, 0)
    except Exception as e:
        print(e)

def add_edge(g, n, m, peso):
    #print("De: ", n, "para: ", m, "peso: ", peso)
    g.add_edge(n, m)
    g[n][m]['weight'] = peso

def add_edge_transfer(g, n, m, peso, nome_jogador):
    #print("De: ", n, "para: ", m, "peso: ", peso)
    g.add_edge(n, m)
    g[n][m]['weight'] = peso
    g[n][m]['jogador'] = nome_jogador

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def get_team_id(connection, nome):
    with connection.cursor() as cursor:
        cursor.execute("SELECT id_time FROM times WHERE nome=%s", (nome))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers(connection, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia WHERE ano=%s", (ano))
        res = cursor.fetchall()
        return res
def get_from_where(connection, id_jogador, ano):
    #print(ano-1)
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, ano))
        res = cursor.fetchone()
        if res:
            return res
        else:
            #buscar no ano anterior
            cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, ano-1))
            res = cursor.fetchone()
            if(res):
                #print("achou em 2018")
                return res
            else:
                #print(id_jogador, ano)
                return (-1,)
def get_weight(connection, id_time1, id_time2, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time1, id_time2, ano))
        res = cursor.fetchone()
        if res:
            return res[0]
        else: #Pode ser que os nomes estejam invertidos nao sei
            cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time2, id_time1, ano))
            res = cursor.fetchone()
            if res:
                return res[0]
            return -1
def get_player_name(connection, id_jogador):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM jogador WHERE id_jogador=%s", (id_jogador))
        res = cursor.fetchone()
        if(res):
            return res
        else:
            return -1
def get_player_id(connection, nome_jogador):
    with connection.cursor() as cursor:
        cursor.execute("SELECT id_jogador FROM jogador WHERE nome=%s", (nome_jogador))
        res = cursor.fetchone()
        if(res):
            return res
        else:
            return -1
def get_player_tag(connection, id_jogador):
    with connection.cursor() as cursor:
        cursor.execute("SELECT tag FROM jogador_tag WHERE id_jogador=%s", (id_jogador))
        res = cursor.fetchone()
        if(res):
            return res
        else:
            return -1

In [ ]:
a = fm.Animation()
g_2019 = nx.DiGraph()
a.rec(g_2019)

transfers = list_all_transfers(connection, 2019)
teams = list_all_teams(connection)

for team in teams:
    add_node(g_2019, team[0])
    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    nome_jogador = get_player_name(connection, transfer[1])[0]
    antigo = get_from_where(connection, transfer[1], transfer[3])[0]
    novo = get_team_name(connection, transfer[2])[0]  
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge_transfer(g_2019, antigo, novo, weight, nome_jogador)
    else:
        continue
        #print(antigo, novo)
        #print(transfer[1])
    
a.rec(g_2019)
a.play()

In [ ]:


a = fm.Animation()
g_2018 = nx.DiGraph()
a.rec(g_2018)

transfers = list_all_transfers(connection, 2018)
teams = list_all_teams(connection)

for team in teams:
    add_node(g_2018, team[0])

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    nome_jogador = get_player_name(connection, transfer[1])[0]
    antigo = get_from_where(connection, transfer[1], transfer[3])[0]
    novo = get_team_name(connection, transfer[2])[0]
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge_transfer(g_2018, antigo, novo, weight, nome_jogador)
    else:
        continue
        #print(antigo, novo)
    
    
a.rec(g_2018)
a.play()


In [ ]:
a = fm.Animation()
g_2017 = nx.DiGraph()
a.rec(g_2017)

transfers = list_all_transfers(connection, 2017)
teams = list_all_teams(connection)

for team in teams:
    add_node(g_2017, team[0])

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    nome_jogador = get_player_name(connection, transfer[1])[0]
    antigo = get_from_where(connection, transfer[1], transfer[3])[0]
    novo = get_team_name(connection, transfer[2])[0]
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge_transfer(g_2017, antigo, novo, weight, nome_jogador)
    else:
        continue
        #print(antigo, novo)
    
    
a.rec(g_2017)
a.play()

In [ ]:
a = fm.Animation()
g_2016 = nx.DiGraph()
a.rec(g_2016)

transfers = list_all_transfers(connection, 2016)
teams = list_all_teams(connection)

for team in teams:
    add_node(g_2016, team[0])

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    nome_jogador = get_player_name(connection, transfer[1])[0]
    antigo = get_from_where(connection, transfer[1], transfer[3])[0]
    novo = get_team_name(connection, transfer[2])[0]
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge_transfer(g_2016, antigo, novo, weight, nome_jogador)
    else:
        continue
        #print(antigo, novo)
    
    
a.rec(g_2016)
a.play()

In [ ]:
#Os grafos agora estão salvos e podemos percorrê-los para comparar as analises sentimentais de cada uma das transferencias com 
# o peso da rivalidade
ano = 2019
counter = 0
dict_analise = {}
for n, m in g_2019.edges:
    try:
        nome_jogador = g_2019.edges[n,m]['jogador']
        id_jogador = get_player_id(connection, nome_jogador)
        tag_twitter = get_player_tag(connection, id_jogador)
        if(tag_twitter == -1):
            continue
            #print("Não achou: ", nome_jogador)
        else:
            tag_twitter = tag_twitter[0]
            #print("Jogador: ", g_2019.edges[n,m]['jogador'], "Time: De:", n, "Para:", m)
            with open("analises/" + tag_twitter + "_" + str(ano) + ".json", "r") as f:
                lista_c = []
                data = json.load(f)
                if(data):
                    counter+=1
                    for i in range(len(data[tag_twitter])):
                        #print(data[tag_twitter][i][1]['compound'])
                        try:
                            c = data[tag_twitter][i][1]['compound']
                            #print(type(c))
                            if(c != 0.0):
                                lista_c.append(c)
                        except Exception as e:
                            continue
                    #print(lista_c)
                    sentimento = sum(lista_c)/len(lista_c)
                    dict_analise[nome_jogador] = [sentimento, g_2019.edges[n,m]['weight']]
    except Exception as e:
        print(e)
print(counter)
print(dict_analise)
filename = "dict_analise2019.json"
print("Salvando...")
try:
    os.remove(filename)
except:
    pass
with open(filename, 'w') as f:
    json.dump(dict_analise, f, indent=2)

In [ ]:
#Os grafos agora estão salvos e podemos percorrê-los para comparar as analises sentimentais de cada uma das transferencias com 
# o peso da rivalidade
ano = 2018
counter = 0
dict_analise_2018 = {}
for n, m in g_2018.edges:
    try:
        nome_jogador = g_2018.edges[n,m]['jogador']
        id_jogador = get_player_id(connection, nome_jogador)
        tag_twitter = get_player_tag(connection, id_jogador)
        if(tag_twitter == -1):
            continue
            #print("Não achou: ", nome_jogador)
        else:
            tag_twitter = tag_twitter[0]
            with open("analises/" + tag_twitter + "_" + str(ano) + ".json", "r") as f:
                lista_c = []
                data = json.load(f)
                if(data):
                    counter+=1
                    for i in range(len(data[tag_twitter])):
                        #print(data[tag_twitter][i][1]['compound'])
                        try:
                            c = data[tag_twitter][i][1]['compound']
                            #print(type(c))
                            if(c != 0.0):
                                lista_c.append(c)
                        except Exception as e:
                            continue
                    #print(lista_c)
                    sentimento = sum(lista_c)/len(lista_c)
                    dict_analise_2018[nome_jogador] = [sentimento, g_2018.edges[n,m]['weight']]
    except Exception as e:
        print(e)
print(counter)
print(dict_analise_2018)
filename = "dict_analise2018.json"
print("Salvando...")
try:
    os.remove(filename)
except:
    pass
with open(filename, 'w') as f:
    json.dump(dict_analise_2018, f, indent=2)

In [ ]:
#Os grafos agora estão salvos e podemos percorrê-los para comparar as analises sentimentais de cada uma das transferencias com 
# o peso da rivalidade
ano = 2017
counter = 0
dict_analise_2017 = {}
for n, m in g_2017.edges:
    try:
        nome_jogador = g_2017.edges[n,m]['jogador']
        id_jogador = get_player_id(connection, nome_jogador)
        tag_twitter = get_player_tag(connection, id_jogador)
        if(tag_twitter == -1):
            continue
            #print("Não achou: ", nome_jogador)
        else:
            tag_twitter = tag_twitter[0]
            with open("analises/" + tag_twitter + "_" + str(ano) + ".json", "r") as f:
                lista_c = []
                data = json.load(f)
                if(data):
                    counter+=1
                    for i in range(len(data[tag_twitter])):
                        #print(data[tag_twitter][i][1]['compound'])
                        try:
                            c = data[tag_twitter][i][1]['compound']
                            #print(type(c))
                            if(c != 0.0):
                                lista_c.append(c)
                        except Exception as e:
                            continue
                    #print(lista_c)
                    sentimento = sum(lista_c)/len(lista_c)
                    dict_analise_2017[nome_jogador] = [sentimento, g_2017.edges[n,m]['weight']]
    except Exception as e:
        print(e)
print(counter)
print(dict_analise_2017)
filename = "dict_analise2017.json"
print("Salvando...")
try:
    os.remove(filename)
except:
    pass
with open(filename, 'w') as f:
    json.dump(dict_analise_2017, f, indent=2)

In [ ]:
#Os grafos agora estão salvos e podemos percorrê-los para comparar as analises sentimentais de cada uma das transferencias com 
# o peso da rivalidade
ano = 2016
counter = 0
dict_analise_2016 = {}
for n, m in g_2016.edges:
    try:
        nome_jogador = g_2016.edges[n,m]['jogador']
        id_jogador = get_player_id(connection, nome_jogador)
        tag_twitter = get_player_tag(connection, id_jogador)
        if(tag_twitter == -1):
            continue
            #print("Não achou: ", nome_jogador)
        else:
            tag_twitter = tag_twitter[0]
            with open("analises/" + tag_twitter + "_" + str(ano) + ".json", "r") as f:
                lista_c = []
                data = json.load(f)
                if(data):
                    counter+=1
                    for i in range(len(data[tag_twitter])):
                        #print(data[tag_twitter][i][1]['compound'])
                        try:
                            c = data[tag_twitter][i][1]['compound']
                            #print(type(c))
                            if(c != 0.0):
                                lista_c.append(c)
                        except Exception as e:
                            continue
                    #print(lista_c)
                    sentimento = sum(lista_c)/len(lista_c)
                    dict_analise_2016[nome_jogador] = [sentimento, g_2016.edges[n,m]['weight']]
    except Exception as e:
        print(e)
print(counter)
print(dict_analise_2016)
filename = "dict_analise2016.json"
print("Salvando...")
try:
    os.remove(filename)
except:
    pass
with open(filename, 'w') as f:
    json.dump(dict_analise_2016, f, indent=2)

TUDO QUE ESTÁ ABAIXO É O ALGORITMO USADO PARA ACHAR AS RIVALIDADES OCULTAS.

In [ ]:
import freeman as fm
teams = list_all_teams(connection)
def colorize_edges(g):
    for n, m in g.edges:
        weight = g.edges[n, m]['weight']
        g.edges[n, m]['color'] = (0, 0, int(255*weight))

g_triads = nx.Graph()
for team in teams:
    add_node(g_triads, team[0])
g_triads = fm.Graph(g_triads)

for a in g_triads.nodes:
    for b in g_triads.nodes:
        if(a == b):
            continue
        else:
            a_id = get_team_id(connection,a)[0]
            b_id = get_team_id(connection,b)[0]
            weight = get_weight(connection, a_id, b_id, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g_triads, a, b, weight)

In [ ]:
a = fm.Animation()
a.rec(g_triads)

dr = 0.4
counter_caso1 = 0
counter_caso2 = 0
counter_caso3 = 0
counter_caso35 = 0
counter_caso4 = 0
while(dr > 0.01):
    print(dr)
    for n,m,l in g_triads.triads():        
        nm_weight = g_triads.edges[n,m]['weight']
        nl_weight = g_triads.edges[n,l]['weight']
        ml_weight = g_triads.edges[m,l]['weight']

        #Caso1 - R R R - nao faz nada
        if(nm_weight >= 0.5) and (nl_weight >= 0.5) and (ml_weight >= 0.5):
            counter_caso1+=1
            continue
        
        #Caso2 - R R X - vira R R dr
        elif(nm_weight >= 0.5) and (nl_weight >= 0.5):
            #Adiciona dr
            ml_weight += dr
            g_triads.edges[m,l]['weight'] = ml_weight
            counter_caso2+=1
        elif(nm_weight >= 0.5) and (ml_weight >= 0.5):
            #Adiciona dr
            nl_weight += dr
            g_triads.edges[n,l]['weight'] = nl_weight
            counter_caso2+=1
        elif(nl_weight >= 0.5) and (ml_weight >= 0.5):
            #Adiciona dr
            nm_weight += dr
            g_triads.edges[n,m]['weight'] = nm_weight
            counter_caso2+=1
            
        #Caso3 - R dx X - vira R dx ddx
        elif((nl_weight >= 0.5) and (ml_weight >= dr)):
            #2 vezes dr porque é da iteração anterior
            #Adiciona ddr
            nm_weight += dr/2
            g_triads.edges[n,m]['weight'] = nm_weight
            counter_caso3+=1

        elif((nm_weight >= 0.5) and (ml_weight >= dr)):
            #Adiciona ddr
            nl_weight += dr/2
            g_triads.edges[n,l]['weight'] = nl_weight
            counter_caso3+=1

        elif((nl_weight >= 0.5) and (nm_weight >= dr)):
            #Adiciona ddr
            ml_weight += dr/2
            g_triads.edges[m,l]['weight'] = ml_weight
            counter_caso3+=1
            
        #Caso3.5 - R ddx X - vira R ddx dddx
        elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
            #2 vezes dr porque é da iteração anterior
            #Adiciona ddr
            nm_weight += dr/4
            g_triads.edges[n,m]['weight'] = nm_weight
            counter_caso35+=1

        elif((nm_weight >= 0.5) and (ml_weight >= dr/2)):
            #Adiciona ddr
            nl_weight += dr/4
            g_triads.edges[n,l]['weight'] = nl_weight
            counter_caso35+=1

        elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
            #Adiciona ddr
            ml_weight += dr/4
            g_triads.edges[m,l]['weight'] = ml_weight
            counter_caso35+=1
             
        #Caso4 - R X X - nao faz nada, quem sabe na proxima iteração
        else:  
            counter_caso4+=1
            continue
    dr = dr/2
    a.rec(g_triads)
    colorize_edges(g_triads)
    g_triads.move('step', iterations=10)

a.rec(g_triads)
print("Caso1: ", counter_caso1)
print("Caso2: ", counter_caso2)
print("Caso3: ", counter_caso3)
print("Caso3.5: ", counter_caso35)
print("Caso4: ", counter_caso4)

a.play()


In [ ]:


teams = list_all_teams(connection)
def colorize_edges(g):
    for n, m in g.edges:
        weight = g.edges[n, m]['weight']
        g.edges[n, m]['color'] = (0, 0, int(255*weight))

g_triads_2018 = nx.Graph()
for team in teams:
    add_node(g_triads_2018, team[0])
g_triads_2018 = fm.Graph(g_triads_2018)

for a in g_triads_2018.nodes:
    for b in g_triads_2018.nodes:
        if(a == b):
            continue
        else:
            a_id = get_team_id(connection,a)[0]
            b_id = get_team_id(connection,b)[0]
            weight = get_weight(connection, a_id, b_id, 2018)
            if(weight == -1):
                weight = 0
            add_edge(g_triads_2018, a, b, weight)

In [ ]:
a = fm.Animation()
a.rec(g_triads_2018)

dr = 0.4
counter_caso1 = 0
counter_caso2 = 0
counter_caso3 = 0
counter_caso35 = 0
counter_caso4 = 0
while(dr > 0.01):
    print(dr)
    for n,m,l in g_triads_2018.triads():        
        nm_weight = g_triads_2018.edges[n,m]['weight']
        nl_weight = g_triads_2018.edges[n,l]['weight']
        ml_weight = g_triads_2018.edges[m,l]['weight']

        #Caso1 - R R R - nao faz nada
        if(nm_weight >= 0.5) and (nl_weight >= 0.5) and (ml_weight >= 0.5):
            counter_caso1+=1
            continue
        
        #Caso2 - R R X - vira R R dr
        elif(nm_weight >= 0.5) and (nl_weight >= 0.5):
            #Adiciona dr
            ml_weight += dr
            g_triads_2018.edges[m,l]['weight'] = ml_weight
            counter_caso2+=1
        elif(nm_weight >= 0.5) and (ml_weight >= 0.5):
            #Adiciona dr
            nl_weight += dr
            g_triads_2018.edges[n,l]['weight'] = nl_weight
            counter_caso2+=1
        elif(nl_weight >= 0.5) and (ml_weight >= 0.5):
            #Adiciona dr
            nm_weight += dr
            g_triads_2018.edges[n,m]['weight'] = nm_weight
            counter_caso2+=1
            
        #Caso3 - R dx X - vira R dx ddx
        elif((nl_weight >= 0.5) and (ml_weight >= dr)):
            #2 vezes dr porque é da iteração anterior
            #Adiciona ddr
            nm_weight += dr/2
            g_triads_2018.edges[n,m]['weight'] = nm_weight
            counter_caso3+=1

        elif((nm_weight >= 0.5) and (ml_weight >= dr)):
            #Adiciona ddr
            nl_weight += dr/2
            g_triads_2018.edges[n,l]['weight'] = nl_weight
            counter_caso3+=1

        elif((nl_weight >= 0.5) and (nm_weight >= dr)):
            #Adiciona ddr
            ml_weight += dr/2
            g_triads_2018.edges[m,l]['weight'] = ml_weight
            counter_caso3+=1
            
        #Caso3.5 - R ddx X - vira R ddx dddx
        elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
            #2 vezes dr porque é da iteração anterior
            #Adiciona ddr
            nm_weight += dr/4
            g_triads_2018.edges[n,m]['weight'] = nm_weight
            counter_caso35+=1

        elif((nm_weight >= 0.5) and (ml_weight >= dr/2)):
            #Adiciona ddr
            nl_weight += dr/4
            g_triads_2018.edges[n,l]['weight'] = nl_weight
            counter_caso35+=1

        elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
            #Adiciona ddr
            ml_weight += dr/4
            g_triads_2018.edges[m,l]['weight'] = ml_weight
            counter_caso35+=1
             
        #Caso4 - R X X - nao faz nada, quem sabe na proxima iteração
        else:  
            counter_caso4+=1
            continue
    dr = dr/2
    a.rec(g_triads_2018)
    colorize_edges(g_triads_2018)
    g_triads_2018.move('step', iterations=10)

a.rec(g_triads_2018)
print("Caso1: ", counter_caso1)
print("Caso2: ", counter_caso2)
print("Caso3: ", counter_caso3)
print("Caso3.5: ", counter_caso35)
print("Caso4: ", counter_caso4)

a.play()


In [ ]:
import pandas as pd
dict_2019 = {}
for n,m in g_triads.edges:
    dict_2019[n,m] = g_triads.edges[n,m]['weight']
df_2019 = pd.DataFrame.from_dict(dict_2019, orient='index')

dict_antigos = {}
for n,m in g_triads_2018.edges:
    dict_antigos[n,m] = g_triads_2018.edges[n,m]['weight']
df_antigos = pd.DataFrame.from_dict(dict_antigos, orient='index')

In [ ]:
df_2019.columns = ['Rivalidade pos algoritmo']
df_2019.loc[df_2019['Rivalidade pos algoritmo'] != 0]

In [ ]:
df_antigos.columns = ['Rivalidade pos algoritmo']
df_antigos.loc[df_antigos['Rivalidade pos algoritmo'] != 0]

In [ ]:
dict_comp = {}
for n,m in g_triads.edges:
    n_id = get_team_id(connection,n)[0]
    m_id = get_team_id(connection,m)[0]
    l_pesos = []
    weight = get_weight(connection, n_id, m_id, 2019)
    if(weight == -1):
        weight = 0
    l_pesos.append(weight)
    l_pesos.append(dict_2019[n,m])
    dict_comp[n,m] = l_pesos
df_comp = pd.DataFrame.from_dict(dict_comp, orient='index')
df_comp.columns = ['Rivalidade antiga', 'Rivalidade pos algoritmo']
df_comp

In [ ]:
dict_comp_antigos = {}
for n,m in g_triads_2018.edges:
    n_id = get_team_id(connection,n)[0]
    m_id = get_team_id(connection,m)[0]
    l_pesos = []
    weight = get_weight(connection, n_id, m_id, 2019)
    if(weight == -1):
        weight = 0
    l_pesos.append(weight)
    l_pesos.append(dict_antigos[n,m])
    dict_comp_antigos[n,m] = l_pesos
df_comp_antigos = pd.DataFrame.from_dict(dict_comp_antigos, orient='index')
df_comp_antigos.columns = ['Rivalidade antiga', 'Rivalidade pos algoritmo']
df_comp_antigos

In [ ]:
export = df_2019.to_json ('./rivalidades2019.json', orient='split')

In [ ]:
export2 = df_antigos.to_json ('./rivalidadesAntigas.json', orient='split')

In [12]:
import freeman as fm
import pandas as pd

In [13]:
def sense(g_triads, dr):
    counter_caso1 = 0
    counter_caso2 = 0
    counter_caso3 = 0
    counter_caso35 = 0
    counter_caso4 = 0
    dr_ini = dr
    while(dr > 0.01):
        print(dr)
        for n,m,l in g_triads.triads():        
            nm_weight = g_triads.edges[n,m]['weight']
            nl_weight = g_triads.edges[n,l]['weight']
            ml_weight = g_triads.edges[m,l]['weight']

            #Caso1 - R R R - nao faz nada
            if(nm_weight >= 0.5) and (nl_weight >= 0.5) and (ml_weight >= 0.5):
                counter_caso1+=1
                continue

            #Caso2 - R R X - vira R R dr
            elif(nm_weight >= 0.5) and (nl_weight >= 0.5):
                #Adiciona dr
                ml_weight += dr
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso2+=1
            elif(nm_weight >= 0.5) and (ml_weight >= 0.5):
                #Adiciona dr
                nl_weight += dr
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso2+=1
            elif(nl_weight >= 0.5) and (ml_weight >= 0.5):
                #Adiciona dr
                nm_weight += dr
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso2+=1

            #Caso3 - R dx X - vira R dx ddx
            elif((nl_weight >= 0.5) and (ml_weight >= dr)):
                #2 vezes dr porque é da iteração anterior
                #Adiciona ddr
                nm_weight += dr/2
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso3+=1

            elif((nm_weight >= 0.5) and (ml_weight >= dr)):
                #Adiciona ddr
                nl_weight += dr/2
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso3+=1

            elif((nl_weight >= 0.5) and (nm_weight >= dr)):
                #Adiciona ddr
                ml_weight += dr/2
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso3+=1

            #Caso3.5 - R ddx X - vira R ddx dddx
            elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
                #2 vezes dr porque é da iteração anterior
                #Adiciona ddr
                nm_weight += dr/4
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso35+=1

            elif((nm_weight >= 0.5) and (ml_weight >= dr/2)):
                #Adiciona ddr
                nl_weight += dr/4
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso35+=1

            elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
                #Adiciona ddr
                ml_weight += dr/4
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso35+=1

            #Caso4 - R X X - nao faz nada, quem sabe na proxima iteração
            else:  
                counter_caso4+=1
                continue
        dr = dr/2

    print("Caso1: ", counter_caso1)
    print("Caso2: ", counter_caso2)
    print("Caso3: ", counter_caso3)
    print("Caso3.5: ", counter_caso35)
    print("Caso4: ", counter_caso4)
    
    dictt = {}
    for n,m in g_triads.edges:
        dictt[n,m] = g_triads.edges[n,m]['weight']
    df = pd.DataFrame.from_dict(dictt, orient='index')
    df.columns = ['Rivalidade pos algoritmo']
    df.loc[df['Rivalidade pos algoritmo'] != 0]
    export = df.to_json ('./rivalidades' +str(dr_ini)+ '.json', orient='split')

    
def sense2(g_triads, dr):
    counter_caso1 = 0
    counter_caso2 = 0
    counter_caso3 = 0
    counter_caso35 = 0
    counter_caso4 = 0
    dr_ini = dr
    while(dr > 0.01):
        print(dr)
        for n,m,l in g_triads.triads():        
            nm_weight = g_triads.edges[n,m]['weight']
            nl_weight = g_triads.edges[n,l]['weight']
            ml_weight = g_triads.edges[m,l]['weight']

            #Caso1 - R R R - nao faz nada
            if(nm_weight >= 0.5) and (nl_weight >= 0.5) and (ml_weight >= 0.5):
                counter_caso1+=1
                continue

            #Caso2 - R R X - vira R R dr
            elif(nm_weight >= 0.5) and (nl_weight >= 0.5):
                #Adiciona dr
                ml_weight += dr
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso2+=1
            elif(nm_weight >= 0.5) and (ml_weight >= 0.5):
                #Adiciona dr
                nl_weight += dr
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso2+=1
            elif(nl_weight >= 0.5) and (ml_weight >= 0.5):
                #Adiciona dr
                nm_weight += dr
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso2+=1

            #Caso3 - R dx X - vira R dx ddx
            elif((nl_weight >= 0.5) and (ml_weight >= dr)):
                #2 vezes dr porque é da iteração anterior
                #Adiciona ddr
                nm_weight += dr/2
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso3+=1

            elif((nm_weight >= 0.5) and (ml_weight >= dr)):
                #Adiciona ddr
                nl_weight += dr/2
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso3+=1

            elif((nl_weight >= 0.5) and (nm_weight >= dr)):
                #Adiciona ddr
                ml_weight += dr/2
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso3+=1

            #Caso3.5 - R ddx X - vira R ddx dddx
            elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
                #2 vezes dr porque é da iteração anterior
                #Adiciona ddr
                nm_weight += dr/4
                g_triads.edges[n,m]['weight'] = nm_weight
                counter_caso35+=1

            elif((nm_weight >= 0.5) and (ml_weight >= dr/2)):
                #Adiciona ddr
                nl_weight += dr/4
                g_triads.edges[n,l]['weight'] = nl_weight
                counter_caso35+=1

            elif((nl_weight >= 0.5) and (nm_weight >= dr/2)):
                #Adiciona ddr
                ml_weight += dr/4
                g_triads.edges[m,l]['weight'] = ml_weight
                counter_caso35+=1

            #Caso4 - R X X - nao faz nada, quem sabe na proxima iteração
            else:  
                counter_caso4+=1
                continue
        dr = dr/2

    print("Caso1: ", counter_caso1)
    print("Caso2: ", counter_caso2)
    print("Caso3: ", counter_caso3)
    print("Caso3.5: ", counter_caso35)
    print("Caso4: ", counter_caso4)
    
    dictt = {}
    for n,m in g_triads.edges:
        dictt[n,m] = g_triads.edges[n,m]['weight']
    df = pd.DataFrame.from_dict(dictt, orient='index')
    df.columns = ['Rivalidade pos algoritmo']
    df.loc[df['Rivalidade pos algoritmo'] != 0]
    export = df.to_json ('./rivalidadesantigas' +str(dr_ini)+ '.json', orient='split')

In [14]:
teams = list_all_teams(connection)
def colorize_edges(g):
    for n, m in g.edges:
        weight = g.edges[n, m]['weight']
        g.edges[n, m]['color'] = (0, 0, int(255*weight))
        
dr_s = 0.1
while(dr_s < 0.9):
    g_triads = nx.Graph()
    for team in teams:
        add_node(g_triads, team[0])
    g_triads = fm.Graph(g_triads)

    for a in g_triads.nodes:
        for b in g_triads.nodes:
            if(a == b):
                continue
            else:
                a_id = get_team_id(connection,a)[0]
                b_id = get_team_id(connection,b)[0]
                weight = get_weight(connection, a_id, b_id, 2019)
                if(weight == -1):
                    weight = 0
                add_edge(g_triads, a, b, weight)
    sense(g_triads, dr_s)
    dr_s += 0.1
    
dr_s = 0.1
while(dr_s < 0.9):
    g_triads_2018 = nx.Graph()
    for team in teams:
        add_node(g_triads_2018, team[0])
    g_triads_2018 = fm.Graph(g_triads_2018)

    for a in g_triads_2018.nodes:
        for b in g_triads_2018.nodes:
            if(a == b):
                continue
            else:
                a_id = get_team_id(connection,a)[0]
                b_id = get_team_id(connection,b)[0]
                weight = get_weight(connection, a_id, b_id, 2018)
                if(weight == -1):
                    weight = 0
                add_edge(g_triads_2018, a, b, weight)
    sense2(g_triads_2018, dr_s)
    dr_s += 0.1

0.1
0.05
0.025
0.0125
Caso1:  24
Caso2:  136
Caso3:  389
Caso3.5:  62
Caso4:  15629
0.2
0.1
0.05
0.025
0.0125
Caso1:  265
Caso2:  293
Caso3:  956
Caso3.5:  72
Caso4:  18714
0.30000000000000004
0.15000000000000002
0.07500000000000001
0.037500000000000006
0.018750000000000003
Caso1:  1279
Caso2:  514
Caso3:  625
Caso3.5:  78
Caso4:  17804
0.4
0.2
0.1
0.05
0.025
0.0125
Caso1:  3269
Caso2:  252
Caso3:  204
Caso3.5:  47
Caso4:  20588
0.5
0.25
0.125
0.0625
0.03125
0.015625
Caso1:  4187
Caso2:  156
Caso3:  0
Caso3.5:  0
Caso4:  20017
0.6
0.3
0.15
0.075
0.0375
0.01875
Caso1:  4187
Caso2:  156
Caso3:  0
Caso3.5:  0
Caso4:  20017
0.7
0.35
0.175
0.0875
0.04375
0.021875
0.0109375
Caso1:  5003
Caso2:  156
Caso3:  0
Caso3.5:  0
Caso4:  23261
0.7999999999999999
0.39999999999999997
0.19999999999999998
0.09999999999999999
0.049999999999999996
0.024999999999999998
0.012499999999999999
Caso1:  5003
Caso2:  156
Caso3:  0
Caso3.5:  0
Caso4:  23261
0.8999999999999999
0.44999999999999996
0.22499999999999998


In [12]:
connection.close()

Error: Already closed